In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [5]:
!pip install --upgrade keras

In [6]:
%cd ./keras-retinanet/

!pip install .

In [7]:
!python setup.py build_ext --inplace

In [8]:
!pip install gdown
!pip install tensorflow-gpu

In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tempfile

import numpy as np
import pandas as pd
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time
import ast
import cv2

from tensorflow import keras
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("GPU info: %s" % tf.test.gpu_device_name())

In [10]:
os.makedirs("./snapshots", exist_ok=True)

In [11]:
!gdown --id 1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R --output ./snapshots/resnet50_csv_10.h5

In [12]:
os.makedirs("./CSOT", exist_ok=True)

In [13]:
df = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
df.head()

In [14]:
df['imageName']=df['video_frame'].astype(str)+'.jpg'
dfFiltered = df[['imageName', 'video_id', 'video_frame', 'annotations']]
#dfFiltered.head()

In [15]:
train_v0=dfFiltered[(dfFiltered['video_id']==0) & (dfFiltered['annotations']!='[]')].reset_index()
train_v1=dfFiltered[(dfFiltered['video_id']==1) & (dfFiltered['annotations']!='[]')].reset_index()
train_v2=dfFiltered[(dfFiltered['video_id']==2) & (dfFiltered['annotations']!='[]')].reset_index()

In [16]:
train_v0.loc[5,'annotations'].count('{')

In [17]:
#Annotation creation
def create_csv(vfold,vname,ifo):
    annots =[]
    #column_names = ['filename','width','height','class','xmin','ymin','xmax','ymax']
    column_names = ['image_name','x_min','y_min','x_max','y_max','class_name']
    TFR = pd.DataFrame(columns=column_names)
    raw_annots = vfold['annotations'].apply(lambda annots : eval(annots)).values
    annots = [annot for sublist in raw_annots for annot in sublist]
    annots = [list(annot.values()) for annot in annots]
    annots = np.array(annots)
    j=0
    for i in range(len(vfold)):
      imageN = vfold.loc[i,'imageName']
      nboxs=vfold.loc[i,'annotations'].count('{')
      for ii in range(0,nboxs):
        #new_row = ['../input/tensorflow-great-barrier-reef/train_images/'+ifo+imageN,1280,720,'class',annots[j+ii,0],annots[j+ii,1],annots[j+ii,2]+annots[j+ii,0],annots[j+ii,3]+annots[j+ii,1]]
        new_row = ['/kaggle/input/tensorflow-great-barrier-reef/train_images/'+ifo+imageN,annots[j+ii,0],annots[j+ii,1],annots[j+ii,2]+annots[j+ii,0],annots[j+ii,3]+annots[j+ii,1],'CSOT']
        TFR.loc[(j+ii)] = new_row
      j+=ii+1 
    TFR.to_csv('./'+vname+'_TFRecord.csv',index=False)
    TFR.head()

In [18]:
#Create annotation csv files
vfold=['train_v0','train_v1','train_v2']
for folders in range(len(vfold)):
    if folders==0: 
        df=train_v0
        vf='video_0/'
    if folders==1: 
        df=train_v1 
        vf='video_1/'
    if folders==2: 
        df=train_v2
        vf='video_2/'
    df.name=vfold[folders] 
    create_csv(df,df.name,vf)

In [19]:
def show_image_objects(image_row):
  
  img_path = image_row.image_name
  box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]

  image = read_image_bgr(img_path)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_box(draw, box, color=(255, 255, 0))

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [20]:
train_0 = pd.read_csv('./train_v0_TFRecord.csv')
test_0 = pd.read_csv('./train_v2_TFRecord.csv')

In [21]:
show_image_objects(train_0.iloc[0])

In [22]:
ANNOTATIONS_FILE = './annotations.csv'
CLASSES_FILE = './classes.csv'

In [23]:
train_0.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [24]:
classes = set(['CSOT'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

In [25]:
!head classes.csv

In [26]:
!head annotations.csv

*Model Training*

In [27]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5' 

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [28]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 200 --epochs 3 csv ./annotations.csv ./classes.csv

In [ ]:
!ls ./snapshots

In [ ]:
model_path = os.path.join('./snapshots', sorted(os.listdir('./snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

In [ ]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [ ]:
THRES_SCORE = 0.6

def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)

In [ ]:
def show_detected_objects(image_row):
  img_path = image_row.image_name
  
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  true_box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]
  draw_box(draw, true_box, color=(255, 255, 0))

  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [ ]:
test_0.head(n=10)

In [ ]:
show_detected_objects(test_0.iloc[0])

In [ ]:
import os
os.chdir(r'./snapshots')
from IPython.display import FileLink
FileLink(r'resnet50_csv_01.h5')
FileLink(r'resnet50_csv_02.h5')
FileLink(r'resnet50_csv_03.h5')